In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left",on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

In [2]:
# count the total number of schools, students, budget and format them appropriately
total_schools = len(school_data_complete["school_name"].unique())
total_students = school_data_complete["student_name"].count()
total_students_formatted = "{:,}".format(total_students)
total_budget = school_data_complete['budget'].unique().sum()
total_budget_formatted = '${:,.2f}'.format(float(total_budget))

# calculate the avg scores
avg_math_score = school_data_complete['math_score'].mean()
avg_reading_score = school_data_complete['reading_score'].mean()
passing_math = (school_data_complete['math_score'] >= 70).sum()/total_students
passing_math_formatted = "{:.2%}".format(passing_math)
passing_reading = (school_data_complete['reading_score'] >= 70).sum()/total_students
passing_reading_formatted = "{:.2%}".format(passing_reading)
passing_overall = ((avg_math_score + avg_reading_score)/2)/100
passing_overall_formatted = "{:.2%}".format(passing_overall)

In [3]:
district_summary_table = pd.DataFrame({"Total Schools": total_schools,
                                       "Total Students": total_students_formatted,
                                       "Total Budget": total_budget_formatted,
                                       "Average Math Score" : avg_math_score,
                                       "Average Reading Score" : avg_reading_score,
                                       "% Passing Math" : passing_math_formatted,
                                       "% Passing Reading" : passing_reading_formatted,
                                       "Overall Passing Rate" : passing_overall_formatted
                                    },index=[0])
district_summary_table

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,80.43%


# School Summary

In [4]:
# group the data by school name and calculate the budget per student
school_data_grouped = school_data_complete.groupby(["school_name"])
school_type = school_data_grouped["type"].unique()
students_per_school = school_data_grouped["school_name"].count()
school_budget = school_data_grouped["budget"].unique()
school_budget_formatted = school_budget.astype('float64').map("${:,.2f}".format)
budget_per_capita = school_budget/students_per_school
budget_per_capita_formatted = budget_per_capita.astype('float64').map("${:.2f}".format)

# get the avg math and reading scores by school
school_avg_math = school_data_grouped["math_score"].sum()/students_per_school
school_avg_reading = school_data_grouped["reading_score"].sum()/students_per_school
number_passing_math_by_school = school_data_grouped['math_score'].apply(lambda val: (val >= 70).sum())
percent_passing_math_by_school = (number_passing_math_by_school/students_per_school).map("{:.2%}".format)
number_passing_reading_by_school = school_data_grouped['reading_score'].apply(lambda val: (val >= 70).sum())
percent_passing_reading_by_school = (number_passing_reading_by_school/students_per_school).map("{:.2%}".format)
percent_passing_overall = (((number_passing_math_by_school+number_passing_reading_by_school)/students_per_school)/2).map("{:.3%}".format)

school_summary_table = pd.DataFrame({"School Type": school_type.str.get(0),
                                     "Total Students": students_per_school,
                                     "Total School Budget": school_budget_formatted,
                                     "Per Student Budget": budget_per_capita_formatted,
                                     "Average Math Score": school_avg_math,
                                     "Average Reading Score": school_avg_reading,
                                     "Percent Passing Math": percent_passing_math_by_school,
                                     "Percent Passing Reading": percent_passing_reading_by_school,
                                     "% Overall Passing Rate": percent_passing_overall})
school_summary_table.sort_values(by='% Overall Passing Rate',ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,95.587%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.27%,97.31%,95.291%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,95.270%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,95.266%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.87%,96.54%,95.204%


In [5]:
school_summary_table.sort_values(by='% Overall Passing Rate').head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.37%,80.22%,73.293%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,73.364%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,73.500%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,73.640%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.31%,79.30%,73.804%


# Math Scores by Grade

In [6]:
ninth_graders = school_data_complete.loc[(school_data_complete["grade"] == "9th"),["school_name",'grade','math_score','reading_score']].groupby('school_name')
tenth_graders = school_data_complete.loc[(school_data_complete["grade"] == "10th"),["school_name",'grade','math_score','reading_score']].groupby('school_name')
eleventh_graders = school_data_complete.loc[(school_data_complete["grade"] == "11th"),["school_name",'grade','math_score','reading_score']].groupby('school_name')
twelfth_graders = school_data_complete.loc[(school_data_complete["grade"] == "12th"),["school_name",'grade','math_score','reading_score']].groupby('school_name')

# get the average math scores per grade by school
ninth_math_scores = ninth_graders['math_score'].sum()/ninth_graders['grade'].count()
tenth_math_scores = eleventh_graders['math_score'].sum()/tenth_graders['grade'].count()
eleventh_math_scores = tenth_graders['math_score'].sum()/eleventh_graders['grade'].count()
twelfth_math_scores = twelfth_graders['math_score'].sum()/twelfth_graders['grade'].count()

# concatenate the series into a DataFrame and rename the columns
math_grade_summary_table = pd.concat([ninth_math_scores, tenth_math_scores, eleventh_math_scores, twelfth_math_scores], axis=1)
math_grade_summary_table.columns = ['9th','10th','11th','12th']


# get the average reading scores per grade by school
ninth_reading_scores = ninth_graders['reading_score'].sum()/ninth_graders['grade'].count()
tenth_reading_scores = eleventh_graders['reading_score'].sum()/tenth_graders['grade'].count()
eleventh_reading_scores = tenth_graders['reading_score'].sum()/eleventh_graders['grade'].count()
twelfth_reading_scores = twelfth_graders['reading_score'].sum()/twelfth_graders['grade'].count()

# concatenate the series into a DataFrame and rename the columns
reading_grade_summary_table = pd.concat([ninth_reading_scores, tenth_reading_scores, eleventh_reading_scores, twelfth_reading_scores], axis=1)
reading_grade_summary_table.columns = ['9th','10th','11th','12th']

math_grade_summary_table


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,78.266344,76.258193,76.492218
Cabrera High School,83.094697,85.607296,80.394191,83.277487
Figueroa High School,76.403037,71.442988,82.369535,77.151369
Ford High School,77.361345,71.594633,83.447648,76.179963
Griffin High School,82.044010,74.549261,94.728532,83.356164
Hernandez High School,77.438495,68.397718,87.217831,77.186567
Holden High School,83.787402,76.798246,92.339806,82.855422
Huang High School,77.027251,71.861799,80.751734,77.225641
Johnson High School,77.187857,75.660147,78.547579,76.863248


In [7]:
reading_grade_summary_table

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,81.729621,80.131095,80.912451
Cabrera High School,83.676136,86.665236,81.456432,84.287958
Figueroa High School,81.198598,74.933159,87.609309,81.384863
Ford High School,80.632653,74.838983,87.305008,80.662338
Griffin High School,83.369193,74.945813,94.141274,84.013699
Hernandez High School,80.866860,72.175224,90.965074,80.857143
Holden High School,83.677165,75.728070,92.223301,84.698795
Huang High School,81.290284,76.534550,86.712899,80.305983
Johnson High School,81.260714,78.710676,82.728715,81.227564


# Scores by School Spending

In [8]:
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"] 

In [9]:
# Get the right columns in the summary table 
trimmed_summary_table = school_summary_table[['School Type']]
trimmed_summary_table.loc[:,"Budget per student"] = budget_per_capita
trimmed_summary_table["Budget per student"] = trimmed_summary_table["Budget per student"].str.get(0).astype('float64')
trimmed_summary_table.reset_index(drop=False)
schools_budgets = trimmed_summary_table.drop(columns='School Type')

# merge tables together to get the right columns
full_df_with_capita = school_data_complete.merge(schools_budgets,on='school_name',how='right')
full_df_with_capita

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,Budget per student
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,655.0
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,655.0
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,655.0
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,655.0
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,655.0
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635,655.0
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635,655.0
7,7,Nicole Baker,F,12th,Huang High School,96,69,0,District,2917,1910635,655.0
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635,655.0
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635,655.0


In [10]:
# add bins based on the spending per student
full_df_with_capita["Spending Ranges (per student)"] = pd.cut(full_df_with_capita["Budget per student"], \
                                                                spending_bins, labels=group_names)
full_df_with_capita

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,Budget per student,Spending Ranges (per student)
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,655.0,$645-675
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,655.0,$645-675
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,655.0,$645-675
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,655.0,$645-675
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,655.0,$645-675
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635,655.0,$645-675
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635,655.0,$645-675
7,7,Nicole Baker,F,12th,Huang High School,96,69,0,District,2917,1910635,655.0,$645-675
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635,655.0,$645-675
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635,655.0,$645-675


In [11]:
# calculate the averages and percentages based on the bins

spending_ranges_grouped = full_df_with_capita.groupby(["Spending Ranges (per student)"])
avg_scores = spending_ranges_grouped['reading_score','math_score'].mean()
bin_sizes = spending_ranges_grouped['size'].count()
number_passing_math_per_bin = spending_ranges_grouped['math_score'].apply(lambda val: (val >= 70).sum())
percent_passing_math_per_bin = number_passing_math_per_bin/bin_sizes
number_passing_reading_per_bin = spending_ranges_grouped['reading_score'].apply(lambda val: (val >= 70).sum())
percent_passing_reading_per_bin = number_passing_reading_per_bin/bin_sizes
percent_passing_overall_per_bin = (number_passing_reading_per_bin+number_passing_math_per_bin)/(bin_sizes*2)

scores_by_school_spending = pd.DataFrame({"Average Math Score": avg_scores['math_score'],
                                          "Average Reading Score": avg_scores['reading_score'],
                                          "% Passing Math": percent_passing_math_per_bin.map("{:.2%}".format),
                                          "% Passing Reading": percent_passing_reading_per_bin.map("{:.2%}".format),
                                          "% Passing Overall": percent_passing_overall_per_bin.map("{:.2%}".format)})
scores_by_school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (per student),,,,,
<$585,83.363065,83.964039,93.70%,96.69%,95.19%
$585-615,83.529196,83.838414,94.12%,95.89%,95.01%
$615-645,78.061635,81.434088,71.40%,83.61%,77.51%
$645-675,77.049297,81.005604,66.23%,81.11%,73.67%


# Scores by School Size

In [12]:
# create new bins for school size

school_size = [0, 1000, 2000, 5000]
size_name = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [13]:
# add the bins to the summary table

full_df_with_capita["School Size"] = pd.cut(full_df_with_capita["size"], \
                                                                school_size, labels=size_name)
full_df_with_capita

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,Budget per student,Spending Ranges (per student),School Size
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
7,7,Nicole Baker,F,12th,Huang High School,96,69,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635,655.0,$645-675,Large (2000-5000)


In [15]:
# calculate the averages and percentages on the data based on school size

school_sizes_grouped = full_df_with_capita.groupby(["School Size"])
avg_scores_by_size = school_sizes_grouped['reading_score','math_score'].mean()
num_students_by_size = school_sizes_grouped['size'].count()
number_passing_math_by_size = school_sizes_grouped['math_score'].apply(lambda val: (val >= 70).sum())
percent_passing_math_by_size = number_passing_math_by_size/num_students_by_size
number_passing_reading_by_size = school_sizes_grouped['reading_score'].apply(lambda val: (val >= 70).sum())
percent_passing_reading_by_size = number_passing_reading_by_size/num_students_by_size
percent_passing_overall_by_size = (number_passing_reading_by_size+number_passing_math_by_size)/(num_students_by_size*2)

scores_by_school_size = pd.DataFrame({"Average Math Score": avg_scores_by_size['math_score'],
                                          "Average Reading Score": avg_scores_by_size['reading_score'],
                                          "% Passing Math": percent_passing_math_by_size.map("{:.2%}".format),
                                          "% Passing Reading": percent_passing_reading_by_size.map("{:.2%}".format),
                                          "% Passing Overall": percent_passing_overall_by_size.map("{:.2%}".format)})
scores_by_school_size



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),83.828654,83.974082,93.95%,96.04%,95.00%
Medium (1000-2000),83.372682,83.867989,93.62%,96.77%,95.19%
Large (2000-5000),77.477597,81.198674,68.65%,82.13%,75.39%


# Score by School Type

In [16]:
# group the data by school type, calculate the averages and percentages

school_types_grouped = full_df_with_capita.groupby(["type"])
avg_scores_by_type = school_types_grouped['reading_score','math_score'].mean()
num_students_by_type = school_types_grouped['size'].count()
number_passing_math_by_type = school_types_grouped['math_score'].apply(lambda val: (val >= 70).sum())
percent_passing_math_by_type = number_passing_math_by_type/num_students_by_type
number_passing_reading_by_type = school_types_grouped['reading_score'].apply(lambda val: (val >= 70).sum())
percent_passing_reading_by_type = number_passing_reading_by_type/num_students_by_type
percent_passing_overall_by_type = (number_passing_reading_by_type+number_passing_math_by_type)/(num_students_by_type*2)

scores_by_school_type = pd.DataFrame({"Average Math Score": avg_scores_by_type['math_score'],
                                          "Average Reading Score": avg_scores_by_type['reading_score'],
                                          "% Passing Math": percent_passing_math_by_type.map("{:.2%}".format),
                                          "% Passing Reading": percent_passing_reading_by_type.map("{:.2%}".format),
                                          "% Passing Overall": percent_passing_overall_by_type.map("{:.2%}".format)})
scores_by_school_type




,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
type,,,,,
Charter,83.406183,83.902821,93.70%,96.65%,95.17%
District,76.987026,80.962485,66.52%,80.91%,73.71%
